### Importación de Tensorflow

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### instalar NLTK
- pip3 install nltk

### Importar datos del data set  

In [3]:
! git clone https://github.com/Duchanoy/SentimentDataSet.git

Cloning into 'SentimentDataSet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [4]:

!ls
%cd SentimentDataSet
!ls

sample_data  SentimentDataSet
/content/SentimentDataSet
LICENSE  neg.txt  pos.txt  README.md


# Pre-procesando los datos 

In [5]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
import pickle
from collections import Counter
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
def create_lexicon(pos,neg,hm_lines,Batches):

    lexicon = []
    with open(pos,'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines*Batches]:
            all_words = word_tokenize(l)
            lexicon += list(all_words)

    with open(neg,'r') as f:
        contents = f.readlines()
        for l in contents[:hm_lines*Batches]:
            all_words = word_tokenize(l)
            lexicon += list(all_words)
    lexicon = [lemmatizer.lemmatize(i) for i in lexicon]
    w_counts = Counter(lexicon)
    print(w_counts)
    l2 = []
    for w in w_counts:
        #print(w_counts[w])
        if 1000 > w_counts[w] > 50:
            l2.append(w)
    print(len(l2))
    return l2

In [0]:
def sample_handling(sample,lexicon,classification,hm_lines,Batch_number):

    featureset = []

    with open(sample,'r') as f:
        contents = f.readlines()
        for l in contents[hm_lines*(Batch_number):hm_lines*(Batch_number+1)]:
            current_words = word_tokenize(l.lower())
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    features[index_value] += 1

            features = list(features)
            featureset.append([features,classification])

    return featureset

In [0]:
def create_feature_sets_and_labels(pos,neg,hm_lines,Nbatch,test_size = 0.1):
    lexicon = create_lexicon(pos,neg,hm_lines,Nbatch)
    test_x=[]
    test_y=[]
    for BN in range(Nbatch):
        features = []
        features += sample_handling('pos.txt',lexicon,[1,0],hm_lines,BN)
        features += sample_handling('neg.txt',lexicon,[0,1],hm_lines,BN)
        random.shuffle(features)
        features = np.array(features)

        testing_size = int(test_size*len(features))

        train_x = list(features[:,0][:-testing_size])
        train_y = list(features[:,1][:-testing_size])
        test_x += list(features[:,0][-testing_size:])
        test_y += list(features[:,1][-testing_size:])
        
        with open('sentiment_set'+str(BN)+'.pickle','wb') as f:
            pickle.dump([np.array(train_x).astype("float32"),np.array(train_y).astype("float32")],f)
        
    with open('sentiment_set'+"test"+'.pickle','wb') as f:
        pickle.dump([np.array(test_x).astype("float32"),np.array(test_y).astype("float32")],f)

    

In [9]:
hm_lines = 2500
Batches=2
create_feature_sets_and_labels('pos.txt','neg.txt',hm_lines=hm_lines,Nbatch=Batches)


Counter({'.': 13134, 'the': 9460, ',': 9400, 'a': 8526, 'and': 5821, 'of': 5706, 'it': 4449, 'to': 3982, 'is': 3328, "'s": 3324, 'that': 2490, 'in': 2457, 'film': 1665, 'but': 1540, 'with': 1471, 'movie': 1459, 'this': 1367, 'for': 1360, 'an': 1235, 'you': 1096, 'be': 880, "n't": 875, 'on': 838, 'not': 781, 'by': 754, 'one': 705, 'are': 695, 'more': 687, 'about': 687, 'ha': 679, 'like': 668, 'at': 655, 'from': 637, 'than': 625, '--': 619, '``': 607, 'have': 602, 'his': 602, 'all': 597, 'i': 540, 'so': 523, "'": 506, 'if': 505, 'make': 498, 'or': 496, 'story': 495, 'character': 450, 'what': 439, 'who': 435, 'there': 431, 'too': 429, 'time': 422, 'just': 418, 'into': 396, 'doe': 385, 'most': 376, 'out': 374, 'much': 368, 'no': 365, 'comedy': 362, 'good': 360, 'even': 356, 'will': 352, 'up': 352, 'can': 338, 'some': 313, 'way': 312, 'performance': 311, 'work': 305, 'he': 295, 'only': 290, 'little': 286, 'their': 277, 'do': 273, 'funny': 268, 'life': 264, 'director': 259, '[': 258, ']': 25

In [10]:
!ls

LICENSE  pos.txt    sentiment_set0.pickle  sentiment_settest.pickle
neg.txt  README.md  sentiment_set1.pickle


### Importación de datos 

In [0]:
with open('sentiment_set0.pickle','rb') as f:
    train_x,train_y=pickle.load(f)
with open('sentiment_settest.pickle','rb') as f:
    test_x,test_y=pickle.load(f)

In [12]:
print(train_x.shape)
print(test_x.shape)

(4500, 389)
(1000, 389)


### Declarando la arquitectura

Generando función 

In [0]:
class DNN_model(object):
  def __init__(self,
               LnInput,
               n_nodes_hl1=500,
               n_nodes_hl2=500,
               n_nodes_hl3=500,
               n_classes=10):
    self.h1LW = tf.Variable(np.random.rand(LnInput, n_nodes_hl1),name="hl1weigths",dtype="float32")
    self.h1LB = tf.Variable(np.random.rand(n_nodes_hl1),name="hl1bias",dtype="float32")
    self.h2LW = tf.Variable(np.random.rand(n_nodes_hl1, n_nodes_hl2),name="hl2weigths",dtype="float32")
    self.h2LB = tf.Variable(np.random.rand(n_nodes_hl2),name="hl2bias",dtype="float32")
    self.h3LW = tf.Variable(np.random.rand(n_nodes_hl2, n_nodes_hl3),name="hl3weigths",dtype="float32")
    self.h3LB = tf.Variable(np.random.rand(n_nodes_hl3),name="hl3bias",dtype="float32")
    self.outW = tf.Variable(np.random.rand(n_nodes_hl3, n_classes),name="outweigths",dtype="float32")
    self.outB = tf.Variable(np.random.rand(n_classes),name="outbias",dtype="float32")
    self.trainable_variables =[self.h1LW,self.h1LB,self.h2LW,self.h2LB,self.h3LW,self.h3LB,self.outW,self.outB]          
  def __call__(self,x): 
      # Declarando la arquitectura

      l1 = tf.add(tf.matmul(x,self.h1LW), self.h1LB)
      l1 = tf.nn.relu(l1)

      l2 = tf.add(tf.matmul(l1,self.h2LW), self.h2LB)
      l2 = tf.nn.relu(l2)

      l3 = tf.add(tf.matmul(l2,self.h3LW), self.h3LB)
      l3 = tf.nn.relu(l3)

      output = tf.matmul(l3,self.outW) + self.outB
      return output

In [0]:
 Sentiment_model = DNN_model(len(train_x[0]), n_classes=2)

Seleccionar un optimizador 

In [0]:
optimizador = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001)

### Definir las metricas a usar

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

### Calculo de gradientes y ajuste 

In [0]:
@tf.function
def train_step(model,tdata, labels):
  with tf.GradientTape() as tape:
    predictions = model(tdata)
    #calculo de una funcion de error 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, predictions))
   
  gradients = tape.gradient(loss, model.trainable_variables)
  capped_grads_and_vars = [(grad,model.trainable_variables[index]) for index, grad in enumerate(gradients)]
  optimizador.apply_gradients(capped_grads_and_vars)
  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(model,tdata, labels):
  predictions = model(tdata)
  t_loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, predictions))

  test_loss(t_loss)
  test_accuracy(labels, predictions)

## función de entrenamiento  y prueba

In [0]:
def fitting(model,dataName,EPOCHS,N_batch,batch_size):
  for epoch in range(EPOCHS):
    
    for NB in range(N_batch):
      i=0
      with open(dataName+str(NB)+'.pickle','rb') as f:
                    train_x,train_y=pickle.load(f)
      while i+batch_size <= len(train_x):
        start = i
        end = i+batch_size
        batch_x = train_x[start:end]
        batch_y = train_y[start:end]
        train_step(model,batch_x,batch_y)
        i+=batch_size

    with open(dataName+'test.pickle','rb') as f:
                    test_x,test_y=pickle.load(f)
    test_step(model,test_x,test_y)
      
    template = 'Epoch {}, Perdida: {}, Exactitud: {}, Perdida de prueba: {}, Exactitud de prueba: {}'
    print(template.format(epoch+1,
                         train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


In [20]:
fitting( Sentiment_model,'sentiment_set',EPOCHS=40,N_batch=2,batch_size=20)

Epoch 1, Perdida: 24276.404296875, Exactitud: 50.222225189208984, Perdida de prueba: 7400.0810546875, Exactitud de prueba: 49.599998474121094
Epoch 2, Perdida: 10818.58203125, Exactitud: 50.411109924316406, Perdida de prueba: 3755.827392578125, Exactitud de prueba: 49.599998474121094
Epoch 3, Perdida: 7374.4453125, Exactitud: 51.122222900390625, Perdida de prueba: 4054.718505859375, Exactitud de prueba: 49.599998474121094
Epoch 4, Perdida: 2469.87060546875, Exactitud: 55.80000305175781, Perdida de prueba: 1315.370849609375, Exactitud de prueba: 59.29999923706055
Epoch 5, Perdida: 1812.155517578125, Exactitud: 59.500003814697266, Perdida de prueba: 1201.27587890625, Exactitud de prueba: 60.79999923706055
Epoch 6, Perdida: 1292.3223876953125, Exactitud: 60.68888854980469, Perdida de prueba: 669.8526000976562, Exactitud de prueba: 67.5999984741211
Epoch 7, Perdida: 977.8981323242188, Exactitud: 63.58888626098633, Perdida de prueba: 413.57244873046875, Exactitud de prueba: 67.5999984741211